# Cross Validation

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

## Train_test_split

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [4]:
%%time
from sklearn.neighbors import KNeighborsClassifier

best_score, best_p, best_k = 0, 0, 0
for k in range(2, 10):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_p, best_k = score, p, k
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best score =", best_score)

Best K = 3
Best P = 4
Best score = 0.9860917941585535
CPU times: user 19.1 s, sys: 251 ms, total: 19.4 s
Wall time: 19.7 s


## Cross Validation

In [5]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([0.99537037, 0.98148148, 0.97685185, 0.97674419, 0.97209302])


1.拟合方式：cross_val_score(knn_clf, X_train, y_train)，默认将 X_train 分割成 3 份，并得到 3 个模型的准确率；如果想将 X_train 分割成 k 份cross_val_score(knn_clf, X_train, y_train, cv=k)；

2.判定条件：score = np.mean(scores)，交叉验证中取 3 个模型的准确率的平均值最高时对应的一组参数作为最终的最优超参数；

In [6]:
%%time
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 10):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_score, best_p, best_k = score, p, k

print("Best K =", best_k)
print("Best P =", best_p)
print("Best score =", best_score)

Best K = 2
Best P = 2
Best score = 0.9851507321274763
CPU times: user 25.5 s, sys: 396 ms, total: 25.9 s
Wall time: 26.6 s


In [7]:
best_knn_clf = KNeighborsClassifier(weights = 'distance', n_neighbors = 2, p = 2)

In [8]:
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

## Grid Search

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)],
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=3)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   24.3s finished


GridSearchCV(cv=3,
             estimator=KNeighborsClassifier(n_neighbors=9, p=5,
                                            weights='distance'),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             verbose=1)


1. “3 folds”：就是指网格搜索使用交叉验证的方式，默认将 X_train 分割成 3 份；如果将 X_train 分割成 k 份：grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=k)；
2. “45 candidates”：k 从 [2, 11) 有 9 个值，p 从 [1, 6) 有 5 个值，一共要对 45 组参数进行搜索；

3. “135 fits”：一共要对 45 组参数进行搜索，每次搜索要训练出 3 个模型，一共要进行 135 此拟合训练；

In [10]:
grid_search.best_score_

0.9833023831631073

In [11]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [12]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975